    '''
    =================================================
    Graded Challenge 2

    Nama  : Putri Arzalya Maharani
    Batch : HCK-013

    Program ini dibuat untuk melakukan analisis data pada penjualan seblak yang menentukan keputusan apakah dapat menambahkan pendapatan dari yang sebelumnya dengan cara melakukan web scrapping hingga perhitungan statistik.
    =================================================
    '''

# WEB SCRAPPING 

Import library yang dibutuhkan:
- Import "time" untuk penggunaan time.sleep(3) yang berfungsi untuk menunda waktu eksekusi ke halaman selanjutnya.
- Import "pandas" dipakai untuk pengerjaan yang berkaitan dengan DataFrame
- Import "BeautifulSoup" untuk mengambil data HTML
- Import "Selenium" untuk menjalankan aktivitas pada web browser secara otomatis

In [147]:
import time
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from selenium import webdriver
from scipy.stats import kendalltau # Uji Korelasi Kendall Tau
from scipy.stats import spearmanr # Uji Korelasi Spearman
from scipy.stats import ttest_ind # T-Test

Setelah import library, langkah selanjutnya yaitu set up browser yang digunakan yaitu Chrome. Chrome sebagai browser yang nantinya akan dijalankan otomatis oleh sistem (saat web scrapping).

- SetUp browser yang digunakan yaitu Chrome

In [12]:
driver = webdriver.Chrome()

Mengakses website Tokopedia melalui request.get untuk memastikan apakah web dapat dibuka atau tidak.

In [13]:
#import requests

#page = requests.get ("https://www.tokopedia.com/search?navsource=&page=1&q=seblak&srp_component_id=02.01.00.00&srp_page_id=&srp_page_title=&st=")
#page

Mengakses website Tokopedia dengan BeautifulSoup dan input "html.parser" untuk mengekstrak informasi seperti judul, paragraf, headings, dan lainnya.

In [14]:
url = "https://www.tokopedia.com/search?navsource=&page=1&q=seblak&srp_component_id=02.01.00.00&srp_page_id=&srp_page_title=&st="
driver.get(url)
html = driver.page_source

soup = BeautifulSoup(html, "html.parser")
print(soup.prettify()[:700])

<html data-rh="lang,translate" lang="id" translate="no">
 <head>
  <title>
   Jual seblak | Tokopedia
  </title>
  <meta charset="utf-8"/>
  <meta content="initial-scale=1, minimum-scale=1, maximum-scale=5, user-scalable=yes, width=device-width" name="viewport"/>
  <link crossorigin="" href="https://assets.tokopedia.net" rel="preconnect"/>
  <link href="https://assets.tokopedia.net" nonce="" rel="dns-prefetch"/>
  <link crossorigin="" href="https://gql.tokopedia.com" rel="preconnect"/>
  <link href="https://gql.tokopedia.com" nonce="" rel="dns-prefetch"/>
  <link crossorigin="" href="https://googleads.g.doubleclick.net" rel="preconnect"/>
  <link href="https://googleads.g.doubleclick.net" no


## Box
Element dari box (yang mencakup nama produk, harga produk, nama penjual, asal kota toko penjual, unit yang terjual, dan rating toko) : 
``` 
<div class="css-1asz3by"
```

### Men-scrapping 10 halaman pada website Tokopedia yang berisi penjualan seblak

In [15]:
#Membuat list kosong sebagai penampung
nama = []
harga = []
penjual = []
kota = []
unit = []
rating = []

#Mengakses beberapa halaman
for nomor_halaman in range (1, 11):
    url = f" https://www.tokopedia.com/search?navsource=&page={nomor_halaman}&q=seblak&srp_component_id=02.01.00.00&srp_page_id=&srp_page_title=&st="
    driver.get(url) #Berhasil mengakses halaman
    time.sleep(3) # Waktu tunda selama 3 detik ketika berganti ke page selanjutnya

    #Mengambil source dari halaman sekarang yang diakses
    html = driver.page_source

    #Beautiful Soup akan digunakan
    soup = BeautifulSoup(html, "html.parser")

    # Mengumpulkan item
    boxes = soup.find_all('div', {'class':'css-uwyh54'})

    #Mengambil Informasi
    for box in boxes:
        
        #Mengambil nama
        elements_nama = box.find('div', {'class':'prd_link-product-name css-3um8ox'})
        nama.append(elements_nama.get_text())

        #Mengambil harga
        elements_harga = box.find('div', {'class':'prd_link-product-price css-h66vau'})
        harga.append(elements_harga.get_text())

        #Mengambil nama penjual
        elements_penjual = box.find('span', {'class':'prd_link-shop-name css-1kdc32b flip'})
        penjual.append(elements_penjual.get_text())

        #Mengambil kota toko
        elements_kota = box.find('span', {'class':'prd_link-shop-loc css-1kdc32b flip'})
        kota.append(elements_kota.get_text())

        #Mengambil banyaknya produk terjual
        kumpulan_unit = box.find('span', {'class':'prd_label-integrity css-1sgek4h'})
        try:
            elements_unit = kumpulan_unit.get_text()
            unit.append(elements_unit)

        except:
            unit.append(0)

        #Mengambil rating produk
        kumpulan_rating = box.find('span', {'class':'prd_rating-average-text css-t70v7i'})
        try:
            elements_rating = kumpulan_rating.get_text()
            rating.append(elements_rating)
        except:
            rating.append(0)

In [18]:
# Mengecek jumlah baris tiap kolom dan hasilnya memiliki jumlah baris yang sama.
len(nama)
len(harga)
len(penjual)
len(kota)
len(unit)
len(rating)

350

Membuat DataFrame yang berisikan kode hasil web scrapping 10 halaman yang kemudian disimpan ke file csv

In [19]:
#Membuat DataFrame

df = pd.DataFrame({'nama produk': nama,
                  'harga produk': harga,
                  'nama penjual': penjual,
                  'kota toko': kota,
                  'produk terjual': unit,
                  'rating produk': rating})  #rating toko

df
df.to_csv("jual_seblak.csv", index=False) #menyimpan ke file csv

# DATA EXPLORATION

In [106]:
df = pd.read_csv("/Users/putriarzalya/Desktop/p0-ftds013-hck-g3-putriarzlya/jual_seblak.csv") # Membuka file csv

df

,nama produk,harga produk,nama penjual,kota toko,produk terjual,rating produk
0,"Seblak Rafael, Seblak Coet Instan Halal",Rp25.000,Brother Meat Shop,Depok,250+ terjual,4.9
1,Seblak Instan,Rp8.000,ROEMAH MOSLEM,Kab. Boyolali,50+ terjual,5.0
2,KERUPUK MENTAH KERUPUK SEBLAK MENTAH 500 GR,Rp12.500,BociKakang,Jakarta Selatan,100+ terjual,4.9
3,Kerupuk Bawang Prima / Krupuk Seblak 500gram,Rp17.500,Parengge Rengge bumbu & rempah,Jakarta Timur,5 terjual,5.0
4,KERUPUK SEBLAK PEDAS DAUN JERUK 100 GR,Rp4.500,BociKakang,Jakarta Selatan,500+ terjual,4.8
...,...,...,...,...,...,...
345,seblak kering pedas 250 gr/seblak kencur/kerup...,Rp9.999,aghniya,Tangerang Selatan,500+ terjual,4.8
346,Gelifood Seblak Instan Kerupuk Mawar Bumbu Ken...,Rp15.000,Lidigeli,Kab. Garut,250+ terjual,4.9
347,keripik beling/kaca 1/2 kg. bumbu seblak daun ...,Rp25.999,aghniya,Tangerang Selatan,250+ terjual,4.9
348,Seblak Instan Pedas Home Made,Rp3.500,the Dhecip,Tangerang Selatan,3rb+ terjual,4.9


Setelah melihat isi dari dataframe, langkah selanjutnya yaitu mengetahui tipe data dari tiap kolom dan juga melihat isi dari kolom apakah terdapat missing value atau tidak.

In [107]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 350 entries, 0 to 349
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   nama produk     350 non-null    object 
 1   harga produk    350 non-null    object 
 2   nama penjual    350 non-null    object 
 3   kota toko       350 non-null    object 
 4   produk terjual  350 non-null    object 
 5   rating produk   350 non-null    float64
dtypes: float64(1), object(5)
memory usage: 16.5+ KB


Mengetahui missing value lebih detail dengan menunjukkan jika "False" = tidak ada missing value dan "True" = ada missing value.

In [108]:
df.isnull().any()

nama produk       False
harga produk      False
nama penjual      False
kota toko         False
produk terjual    False
rating produk     False
dtype: bool

In [109]:
df.isnull().sum() # Mengetahui jumlah missing value

nama produk       0
harga produk      0
nama penjual      0
kota toko         0
produk terjual    0
rating produk     0
dtype: int64

Selanjutnya, mengetahui nilai yang unik pada setiap kolom serta tipe data yaitu int64.

In [110]:
df.nunique()

nama produk       100
harga produk       69
nama penjual       68
kota toko          21
produk terjual     34
rating produk      11
dtype: int64

- Mengetahui isi/nilai dari kolom "rating produk".
- Dengan menggunakan value_counts dapat mengetahui isi/value dari suatu kolom sehingga hal ini berlaku di kolom lainnya

In [111]:
df['rating produk'].value_counts

<bound method IndexOpsMixin.value_counts of 0      4.9
1      5.0
2      4.9
3      5.0
4      4.8
      ... 
345    4.8
346    4.9
347    4.9
348    4.9
349    4.8
Name: rating produk, Length: 350, dtype: float64>

# DATA CLEANING

- Memformat teks agar sesuai dengan standar "title case" seperti nama produk, nama nama penjual, dan asal kota toko penjual.
- Menghilangkan "Rp" , "+ terjual" , "rb" agar dapat dilakukan perhitungan statistik

In [112]:
# Mengatur ulang format teks
df['nama produk'] = df['nama produk'].apply(lambda x: x.title())
df['nama penjual'] = df['nama penjual'].apply(lambda x: x.title())
df['kota toko'] = df['kota toko'].apply(lambda x: x.title())

# Menghilangkan "Rp" & "." agar dapat dilakukan perhitungan 
df['harga produk'] = df['harga produk'].str.replace('Rp', '').str.replace('.', '')

# Menghilangkan "+ terjual" dalam produk terjual
df['produk terjual'] = df['produk terjual'].str.replace('+', '').str.replace('terjual', '').str.replace('rb', '000')

df

,nama produk,harga produk,nama penjual,kota toko,produk terjual,rating produk
0,"Seblak Rafael, Seblak Coet Instan Halal",25000,Brother Meat Shop,Depok,250,4.9
1,Seblak Instan,8000,Roemah Moslem,Kab. Boyolali,50,5.0
2,Kerupuk Mentah Kerupuk Seblak Mentah 500 Gr,12500,Bocikakang,Jakarta Selatan,100,4.9
3,Kerupuk Bawang Prima / Krupuk Seblak 500Gram,17500,Parengge Rengge Bumbu & Rempah,Jakarta Timur,5,5.0
4,Kerupuk Seblak Pedas Daun Jeruk 100 Gr,4500,Bocikakang,Jakarta Selatan,500,4.8
...,...,...,...,...,...,...
345,Seblak Kering Pedas 250 Gr/Seblak Kencur/Kerup...,9999,Aghniya,Tangerang Selatan,500,4.8
346,Gelifood Seblak Instan Kerupuk Mawar Bumbu Ken...,15000,Lidigeli,Kab. Garut,250,4.9
347,Keripik Beling/Kaca 1/2 Kg. Bumbu Seblak Daun ...,25999,Aghniya,Tangerang Selatan,250,4.9
348,Seblak Instan Pedas Home Made,3500,The Dhecip,Tangerang Selatan,3000,4.9


Menghapus baris yang mengandung nilai yang sama di semua kolom dari DataFrame.

In [113]:
df.drop_duplicates(inplace=True) #inplace=True agar hasil setelah drop duplicates otomatis tersimpan di df.

df

,nama produk,harga produk,nama penjual,kota toko,produk terjual,rating produk
0,"Seblak Rafael, Seblak Coet Instan Halal",25000,Brother Meat Shop,Depok,250,4.9
1,Seblak Instan,8000,Roemah Moslem,Kab. Boyolali,50,5.0
2,Kerupuk Mentah Kerupuk Seblak Mentah 500 Gr,12500,Bocikakang,Jakarta Selatan,100,4.9
3,Kerupuk Bawang Prima / Krupuk Seblak 500Gram,17500,Parengge Rengge Bumbu & Rempah,Jakarta Timur,5,5.0
4,Kerupuk Seblak Pedas Daun Jeruk 100 Gr,4500,Bocikakang,Jakarta Selatan,500,4.8
...,...,...,...,...,...,...
320,Seblak Makroni Pedas Gurih Ekstra Daun Jeruk 1...,5000,Seblak Putra Bandung,Kab. Bandung,100,4.8
321,Kerupuk Seblak Maknyik,18000,Dilladystore,Malang,60,4.9
322,Kerupuk Seblak Kencur Pedas 100 Gram,5000,Keripikususkrisis,Kab. Gresik,60,4.9
323,Kerupuk Seblak Kering Sebring Pedas Daun Jeruk...,4500,Pedas Gilaa,Surabaya,500,4.9


Merubah tipe data ke yang seharusnya
- harga produk dari Object menjadi Integer
- produk terjual dari Object menjadi Integer

In [114]:
# Merubah tipe data
df['harga produk'] = df['harga produk'].astype(int)
df['produk terjual'] = df['produk terjual'].astype(int)

df.dtypes

nama produk        object
harga produk        int64
nama penjual       object
kota toko          object
produk terjual      int64
rating produk     float64
dtype: object

Selanjutnya setelah cleaning data selesai, hasil cleaning data tersebut disimpan kembali dalam bentuk csv

In [115]:
df.to_csv("jual_seblak_clean.csv", index=False) #menyimpan ke file csv

# Business Understanding/Problem Statement

### SMART FRAMEWORK

Case : Saya memiliki pekerjaan tetap akan tetapi saya merasa harus memiliki pendapatan tambahan diluar pekerjaan saya karena keperluan saya yang semakin banyak oleh karena itu saya memilih sebagai dropshipper dimana saya mencari supplier seblak di toko kemudian saya menjualnya lagi di Tokopedia dengan sedikit perubahan harga yang lebih mahal dari yang saya beli di toko.

- Spesific : Mendapatkan pendapatan tambahan dengan bekerja sebagai dropshipper yang kemudian produk tersebut dijual di Tokopedia.

- Measureable : Menambahkan pendapatan dari yang sebelumnya. Dari pendapatan awalnya yaitu Rp5.000.000 menjadi Rp6.000.000,00 karena melakukan penjualan seblak di Tokopedia (sebagai dropshipper). Metriks yang digunakan yaitu Gross Margin dimana mengukur efisiensi dalam menghasilkan keuntungan (keuntungan yang dimaksud sebagai pendapatan tambahan).

- Achievable : Berdasarkan analisis data yang diperoleh melalui web scrapping Tokopedia yang berisi penjualan seblak, dapat dikatakan melakukan penjualan seblak di Tokopedia bisa mendapatkan pendapatan tambahan hingga total pendapatan sebulan menjadi Rp6.000.000,00.

- Relevant : Melakukan penjualan seblak di Tokopedia (sebagai dropshipper) dapat mempengaruhi penambahan pendapatan kita dari sebelumnya.

- Time Bound : Waktu yang dibutuhkan untuk melakukan ini 2 bulan. Dimana 1 bulan pertama mengecek hal-hal yang diperlukan sebelum dilakukannya. 1 bulan selanjutnya yaitu mengeksekusikan apa yang direncanakan dan memperoleh pendapatan tambahan hingga total pendapatan bisa tercapai sebesar Rp6.000.000,00.

kesimpulan: melakukan penjualan seblak di Tokopedia (sebagai dropshipper) dengan tujuan untuk meningkatkan pendapatan tambahan dimana dilaksanakan dalam waktu 2 bulan.

# Data Analysis

#### Menghitung Rata-rata, median, standar deviasi, skewness, dan kurtosis dari kolom Harga, jumlah produk terjual, dan rating.

In [133]:
# Mean,Median,Standar Deviasi dari kolom "harga produk" menggunakan method .describe
desc = df['harga produk'].describe()
print(desc)

# Menghitung skewness menggunakan method .skew() dari kolom "harga produk"
skewness = df['harga produk'].skew()
print(f'Nilai skewness yang diperoleh dari kolom harga produk yaitu {skewness}')

# Menghitung kurtosis menggunakan method .kurtosis() dari kolom "harga produk"
kurtosis_value = df['harga produk'].kurtosis()
print(f'Nilai kurtosis yang diperoleh dari kolom harga produk yaitu {kurtosis_value}')

count       100.000000
mean      22103.110000
std       27151.964907
min        3000.000000
25%       10000.000000
50%       15500.000000
75%       25000.000000
max      199999.000000
Name: harga produk, dtype: float64
Nilai skewness yang diperoleh dari kolom harga produk yaitu 4.745333661402658
Nilai kurtosis yang diperoleh dari kolom harga produk yaitu 26.96612614841103


Dikarenakan nilai skewness >= 0.5 maka digunakan teknik Tukey's Rule untuk menentukan adanya outlier atau tidak dengan rumus berikut:
- IQR = Q3-Q1 = 25.000 - 10.000 = 15.000 
- Batas Bawah : 10.000 - 1.5 * 15.000 = -12.500
- Batas Atas : 25.000 + 1.5 * 15.000 = 2.500 (Adanya outlier karna terdapat harga yang diatas Rp2.500)

In [134]:
# Mean,Median,Standar Deviasi dari kolom "produk terjual" menggunakan method .describe
desc = df['produk terjual'].describe()
print(desc)

# Menghitung skewness menggunakan method .skew() dari kolom "produk terjual"
skewness = df['produk terjual'].skew()
print(f'Nilai skewness yang diperoleh dari kolom produk terjual yaitu {skewness}')

# Menghitung kurtosis menggunakan method .kurtosis() dari kolom "produk terjual"
kurtosis_value = df['produk terjual'].kurtosis()
print(f'Nilai kurtosis yang diperoleh dari kolom produk terjual yaitu {kurtosis_value}')

count      100.000000
mean       622.910000
std       1796.514156
min          0.000000
25%         30.000000
50%         90.000000
75%        250.000000
max      10000.000000
Name: produk terjual, dtype: float64
Nilai skewness yang diperoleh dari kolom produk terjual yaitu 4.54484893926742
Nilai kurtosis yang diperoleh dari kolom produk terjual yaitu 21.120739099843302


Dikarenakan nilai skewness >= 0.5 maka digunakan teknik Tukey's Rule untuk menentukan adanya outlier atau tidak dengan rumus berikut:
- IQR = Q3-Q1 = 250 - 30 = 220
- Batas Bawah = 30 - 1.5 * 220 = -300
- Batas Atas = 250 + 1.5 * 220 = 580 (Adanya outlier karna terdapat produk yang terjual melebihi 580)

In [130]:
# Mean,Median,Standar Deviasi dari kolom "rating produk" menggunakan method .describe
desc = df['rating produk'].describe()
print(desc)

# Menghitung skewness menggunakan method .skew() dari kolom "rating produk"
skewness = df['rating produk'].skew()
print(f'Nilai skewness yang diperoleh dari kolom rating produk yaitu {skewness}')

# Menghitung skewness menggunakan method .kurtosis() dari kolom "rating produk"
kurtosis_value = df['rating produk'].kurtosis()
print(f'Nilai kurtosis yang diperoleh dari kolom rating produk yaitu {kurtosis_value}')


count    100.000000
mean       4.729000
std        0.856266
min        0.000000
25%        4.800000
50%        4.900000
75%        5.000000
max        5.000000
Name: rating produk, dtype: float64
Nilai skewness yang diperoleh dari kolom rating produk yaitu -5.215919749049569
Nilai kurtosis yang diperoleh dari kolom rating produk yaitu 26.88664515288853


Dikarenakan nilai skewness < 0.5 maka digunakan teknik Z-Score dengan rumus berikut:
- Batas Bawah = mean - 3*std = 4.7 - (3*0.8) = 2.3
- Batas Atas = mean + 3*std = 4.7 + (3*0.8) = 7.1 (Tidak ada outlier karena tidak ada produk dengan rating diatas 7.1)

#### Potensi Minimum dan Maksimum Pendapatan Menjual Produk Seblak

Untuk menghitung Minimum dan Maksimum Pendapatan, cara yang dilakukan yaitu:
- Menghitung nilai rata-rata dan standar deviasi pendapatan dari penjualan seblak
- Menghitung confidence interval (tingkat kepercayaan) = Mean ± Z * (Standard Deviation / √(Sample Size)). Jika confidence interval 95% maka diperoleh Z yaitu 1.96.
- Menghitung lower value(minimum) dan upper value(maksimum)

In [119]:
# Menghitung Pendapatan
pendapatan = df['harga produk'] * df['produk terjual']

# Menghitung Rata-Rata dan Standar Deviasi pendapatan
mean_pendapatan = np.mean(pendapatan)
std_pendapatan = np.std(pendapatan)

confidence_level = 0.95
z_score = 1.96

# Menghitung Confidence Interval
error_margin = z_score * (std_pendapatan / np.sqrt(len(pendapatan)))
print(f'error margin sebesar {error_margin}')
nilai_minimum = mean_pendapatan - error_margin
nilai_maksimum = mean_pendapatan + error_margin

print(f'Nilai potensi minimum pendapatan dari penjualan seblak yaitu {nilai_minimum}')
print(f'Nilai potensi maksimum pendapatan dari penjualan seblak yaitu {nilai_maksimum}')

error margin sebesar 7358303.064450381
Nilai potensi minimum pendapatan dari penjualan seblak yaitu 3084351.7855496183
Nilai potensi maksimum pendapatan dari penjualan seblak yaitu 17800957.91445038


- Margin of error (error_margin) merupakan pandangan tentang batas atas dan bawah dari perkiraan yang dihasilkan dari sampel data.
- Berdasarkan hasil Confidence Interval 95%, diperoleh:
- Potensi pendapatan minimum yang diperoleh dari penjualan seblak yaitu sebesar Rp3.084.351,78
- Potensi pendapatan maksimum yang diperoleh dari penjualan seblak yaitu sebesar Rp17.800.957,91

#### Harga Barang di Jabodetabek dan diluar Jabodetabek

- Dalam kasus ini digunakan hipotesis, yaitu:
- H0 : tidak ada perbedaan signifikan antara harga barang di Jabodetabek dan diluar Jabodetabek
- H1 : ada perbedaan signifikan antara harga barang di Jabodetabek dan diluar Jabodetabek

- Jenis hipotesis yang digunakan dalam kasus ini yaitu berupa membandingkan harga seblak di Jabodetabek dan diluar jabodetabek maka menggunakan uji hipotesis dua sampel independen.

In [120]:
# Mengelompokkan berdasarkan wilayah Jabodetabek dan Non-Jabodetabek
jabodetabek = '|'.join(['Jakarta', 'Bogor', 'Depok', 'Tangerang', 'Bekasi'])
# Membuat kolom baru berdasarkan hipotesis
df['zona'] = np.where(df['kota toko'].str.contains(jabodetabek, case=False), 1, 2)

df

,nama produk,harga produk,nama penjual,kota toko,produk terjual,rating produk,zona
0,"Seblak Rafael, Seblak Coet Instan Halal",25000,Brother Meat Shop,Depok,250,4.9,1
1,Seblak Instan,8000,Roemah Moslem,Kab. Boyolali,50,5.0,2
2,Kerupuk Mentah Kerupuk Seblak Mentah 500 Gr,12500,Bocikakang,Jakarta Selatan,100,4.9,1
3,Kerupuk Bawang Prima / Krupuk Seblak 500Gram,17500,Parengge Rengge Bumbu & Rempah,Jakarta Timur,5,5.0,1
4,Kerupuk Seblak Pedas Daun Jeruk 100 Gr,4500,Bocikakang,Jakarta Selatan,500,4.8,1
6,Seblak Instan Pedas Home Made,3500,The Dhecip,Tangerang Selatan,3000,4.9,1
7,Sambal Seblak By Mamang Rafael Original Halal ...,39000,Beauty Deluxe,Jakarta Barat,18,5.0,1
8,Kerupuk Seblak Rafael Pedas / Seblak Mawar Ped...,15000,Duo Bocil Snack,Depok,80,5.0,1
9,Kylafood Seblak Cup,11000,Kylafood Jakarta,Jakarta Selatan,100,5.0,1
12,Seblak Instant Termurah,4450,Pusat Cuankie & Cemilan Frozen,Jakarta Timur,10000,4.8,1


### Uji T-Test

- Uji T-Test dilakukan karena uji ini merupakan uji inferensial dan juga karena 'assuming' data ini terdistribusi secara normal. 

In [143]:
uniq_values = df['kota toko'].unique()

uniq_values

array(['Depok', 'Kab. Boyolali', 'Jakarta Selatan', 'Jakarta Timur',
       'Tangerang Selatan', 'Jakarta Barat', 'Surakarta', 'Bandung',
       'Kab. Garut', 'Kab. Bandung', 'Jakarta Pusat', 'Surabaya',
       'Tangerang', 'Cimahi', 'Kab. Bekasi', 'Tasikmalaya', 'Semarang',
       'Kab.Ciamis', 'Malang', 'Kab. Gresik', 'Kab. Tangerang'],
      dtype=object)

In [148]:
# Membagi berdasarkan wilayah
toko_jabodetabek = ['Jakarta Barat', 'Jakarta Timur', 'Jakarta Selatan', 'Jakarta Pusat', 'Bogor', 'Depok', 'Tangerang Selatan', 'Bekasi', 'Kab.Tangerang']

produk_jabodetabek = df[df['kota toko'].isin(toko_jabodetabek)]
produk_non_jabodetabek = df[~df['kota toko'].isin(toko_jabodetabek)]

# Menghitung mean
mean_produk_jabodetabek = produk_jabodetabek['harga produk'].mean()
mean_produk_non_jabodetabek = produk_non_jabodetabek['harga produk'].mean()


# Uji T-Test
t_statistic, p_value = ttest_ind(produk_jabodetabek['harga produk'], produk_non_jabodetabek['harga produk'])
print("T-Statistic:", t_statistic)
print("P-Value:", p_value)
if p_value < 0.05:
    print('Korelasi signifikan. H0 berhasil ditolak.')
else:
    print('Tidak ada korelasi signifikan. H0 gagal ditolak.')

T-Statistic: -1.9747663402862066
P-Value: 0.05110852086579032
Tidak ada korelasi signifikan. H0 gagal ditolak.


Dipilihnya uji korelasi Spearman karena uji ini menguji 2 variabel dengan data yang salah satunya bisa berdata ordinal atau nominal dengan syarat bahwa distribusi berjalan secara normal. Uji korelasi ini dilakukan karena "assuming" bahwa distribusi secara normal. 

In [140]:
# Spearmanr
korelasi_koefisien, p_value = spearmanr(df['harga produk'], df['zona'])
print(f'Nilai koefisien spearman: {korelasi_koefisien}')
print(f'P-value: {p_value}')

if p_value < 0.05:
    print('Korelasi signifikan. H0 berhasil ditolak.')
else:
    print('Tidak ada korelasi signifikan. H0 gagal ditolak.')

Nilai koefisien spearman: 0.02978499118253409
P-value: 0.7686276427613845
Tidak ada korelasi signifikan. H0 gagal ditolak.


##### - Pengujian apakah pembeli cenderung menyukai produk yang harganya murah atau tidak dapat dilakukan dengan uji korelasi antara harga produk dan rating.

- Dalam kasus ini digunakan hipotesis, yaitu:
- H0 : tidak ada perbedaan signifikan antara harga barang dengan rating toko
- H1 : ada perbedaan signifikan antara harga barang dengan rating toko

- Jenis hipotesis yang digunakan dalam kasus ini yaitu berupa membandingkan harga seblak dengan rating toko maka menggunakan uji hipotesis dua sampel independen.

Dipilihnya uji korelasi Kendall Tau karena data yang terdistribusi tidak normal dan untuk uji Kendall Tau, bentuk data tidak perlu continous ataupun diskret.

In [141]:
# Kendall Tau
korelasi_koefisien, p_value = kendalltau(df['harga produk'], df['rating produk'])
print(f'Nilai koefisien Kendall Tau: {korelasi_koefisien}')
print(f'P-value: {p_value}')

if p_value < 0.05:
    print('Korelasi signifikan. H0 berhasil ditolak.')
else:
    print('Tidak ada korelasi signifikan. H0 gagal ditolak.')

Nilai koefisien Kendall Tau: 0.13002461673150478
P-value: 0.08857558640767328
Tidak ada korelasi signifikan. H0 gagal ditolak.


# CONCLUSION:

- Berdasarkan perhitungan, diperoleh rata-rata, median, standar deviasi yang tertera diatas. Dengan method .describe() juga dapat mengetahui nilai kuartil 1,2,3. 

- Pada harga produk, jumlah produk yang terjual, rating terjadi adanya outlier karena terdapat harga, jumlah produk terjual, rating yang melebihi batas atas. Maka dari itu distribusinya tidak berjalan normal, hal ini diperkuat dengan data dari nilai skewness. 

- Nilai skewness pada "harga produk" dan "jumlah produk yang terjual" bernilai positif, hal ini menunjukkan bahwa ekor distribusi berada di sebelah nilai kanan terbanyak berarti sebagian besar distribusi berada di nilai rendah. Sedangkan pada "rating" bernilai negatif yang menunjukan bahwa ekor distribusi berada disebelah kiri yang sebagian besar nilai berada di sisi kanan kurva maka nilai rata-rata akan lebih kecil dari median. Nilai skewness yang menunjukan terdistribusinya normal yaitu 0.

- Nilai kurtosis pada "harga produk", "jumlah produk yang terjual", "rating" dapat dikatakan jauh dari distribusi normal. Nilai kurtosis yang dapat dianggap berdistribusi normal yaitu mendekati 3. Jika nilai kurtosis semakin jauh dari nilai ideal berarti penyebarannya tidak merata.

- Dengan menjual seblak di Tokopedia (sebagai dropshipper) dapat dikatakan bahwa potensi minimum pendapatan yang diperoleh sebesar Rp3.084.351,78 dan potensi maksimumnya sebesar Rp17.800.957,91. Berdasarkan kedua potensi pendapatan yang diperoleh, hal ini sangat jelas bahwa dengan melakukan penjualan seblak di Tokopedia akan menambahkan pendapatan diluar pendapatan utama sehinggal hal ini layak dilakukan.

- Membandingkan harga di daerah Jabodetabek dan diluar Jabodetabek karena harga bahan baku setiap daerah tentunya berbeda-beda. Ternyata tidak ada korelasi yang signifikan antara harga produk dengan daerah. Hal tersebut karena kita berjualan di Tokopedia yang sebagaimana online jadi tentunya tidak dipengaruhi daerah.

- Berdasarkan uji T-Test, hasil tidak berkorelasi signifikan. Dapat dikatakan bahwa hubungan antar dua variabel yaitu daerah (jabodetabek dan non jabodetabek) dengan harga produk tidak saling berkaitan. Harga produk di jabodetabek dan non jabodetabek tidak memiliki korelasi signifikan tetapi karena nilai p-value sebesar 0.051 yang dimana angka tersebut mendekati 0.5 artinya tidak berkorelasi signifikan secara lemah.

- Pengujian mengenai pembeli cenderung membeli produk yang murah dilakukan antara harga produk dengan rating toko. Hasil dari p-value menunjukan tidak ada korelasi signifikan. Hal tersebut karena rating toko tidak mempengaruhi pembeli dalam membeli produk yang lebih murah atau tidak. Rating toko hanya untuk melihat apakah toko tersebut trusted atau tidak.

- Perlu diingat kembali bahwa ketidaksignifikan tidak berarti bahwa tidak ada hubungan sama sekali antara kedua variabel tersebut. Hal ini hanya menunjukan bahwa kita tidak memiliki cukup bukti dari sampel yang dianalisis untuk mengatakan bahwa ada hubungan yang signifikan antara kedua variabel.

- Berdasarkan analisis SMART yang dimana goalsnya merupakan pendapatan yang dapat bertambah dari sebelumnya dengan melakukan penjualan seblak di Tokopedia telah sesuai dengan hasil analisis data yang diperoleh melalui web scrapping hingga perhitungan statistik.